# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:**

    Matthew Beaulieu
    
    Yousef Fadila
    
    Meng Li
    
    Monica Tlachac
    
---

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [3]:
import pandas as pd
import matplotlib
import warnings
import json
import numpy as np
import wikiwords
from tweepy import Stream
from tweepy.streaming import StreamListener
import pymongo
import tweepy
import re
from prettytable import PrettyTable

warnings.simplefilter(action = "ignore", category = FutureWarning)
%matplotlib inline
#---------------------------------------------
# Define a Function to Login Twitter API
#
# XXX  - DON'T COMMIT TRUE CREDINTIALS TO GITHUB
CONSUMER_KEY = 'k01qrrsDSOHdjUObbt1pWjhld'
CONSUMER_SECRET ='FTBSa4UU5Nxaw00hNPJQvoAf60HinJM87GfQpuFM0YsVKDdoJD'
OAUTH_TOKEN = '4229853605-mSgVKfVSZ9BSN7GbPgvUIPOT1Xl6F3nA2ynKMvy'
OAUTH_TOKEN_SECRET = 'jyHtNVKGmqNw4HyBLMYG5i9BT3junKGVFnY6CE2tjJRFW'

# another set of auth to use when limit is hit.
#CONSUMER_KEY = '7n45AnattFU5sb3MpbiwIoNYF'
#CONSUMER_SECRET ='VtnEmlR99dG3hQ8FP8xb1TnomTAjSGkQYDJVhGa28IDPDEA2Cf'
#OAUTH_TOKEN = '2497861306-W7d3EXegd4AY6jYt9ZaEz5rq5ZtguAAtp6KbseS'
#OAUTH_TOKEN_SECRET = 'uv71S6pJwPibOFLA8WetoxWRb0nfqcQPK8f3ts30nEfQT'

# mango db auth.
MANGODB_SERVER = 'ds033046.mlab.com'
MANGODB_PORT = 33046
MANGODB_USER = 'ds501u'
MANGODB_PASS = 'ds501p'
MANGODB_NAME = 'ds501case1'
MANGODB_COLLECTION = 'tweets'
GEOBOX_USA = [-125,25.1,-60.5,49.1]
us_states = ["WA" , "DE" , "WI" , "WV" , "HI" , "FL" , "WY" , "NH" , "NJ" , "NM" , "TX" , "LA" , "NC" , "ND" , "NE" , "TN" , "NY" , "PA" , "CA" , "NV" , "VA" , "CO" , "AK" , "AL" , "AR" , "VT" , "IL" , "GA" , "IN" , "IA" , "OK" , "AZ" , "ID" , "CT" , "ME" , "MD" , "MA" , "OH" , "UT" , "MO" , "MN" , "MI" , "RI" , "KS" , "MT" , "MS" , "SC" , "KY" , "OR" , "SD" ] 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

db_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db = db_client[MANGODB_NAME]
db.authenticate(MANGODB_USER, MANGODB_PASS)
tweets_Collection = db[MANGODB_COLLECTION]
#----------------------------------------------

In [1]:
class MyListener(StreamListener):
    def __init__(self, max_count = 100):
        self.count = 0
        self.max_count = max_count   
        self.collection = db[MANGODB_COLLECTION]

    def on_data(self, data):
        try:
            if self.count >= self.max_count:
                return False
            # the question require output to file. so output to file and save in mangoDB
            with open('data/hillary.json', 'a') as f:
                f.write(data)
            self.count = self.count + 1
            datajson = json.loads(data)
            self.collection.insert_one(datajson)
            return True
        except BaseException as e:
            print(e)
        return True
 
    def on_error(self, status):
        print(status)
        return True
    
api = tweepy.API(auth)
limit = 20000
twitter_stream = Stream(auth, MyListener(limit))
twitter_stream.filter(track=['@HillaryClinton'], languages=["en"], locations=GEOBOX_USA)
print("streaming done")


NameError: name 'StreamListener' is not defined

### Report some statistics about the tweets you collected 

* The topic of interest: Hillary Clinton                   


* The total number of tweets collected:  17000

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

**Getting Text from Tweets** 

* Get the text column of the tweet df
* make it lower case
* remove http links and non alphanum characters

In [5]:
################# RE WRITE THIS - DF is not defined in this context. define it or loop over database/file. your coice
lst = []
with open('data/hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lst.append(tweet)

df = pd.DataFrame(lst)    
text = df.text.str.lower()

for i,t in enumerate(text):
    t = re.sub(r"(?:\@|https?\://)\S+", "", t)
    new = ''.join(e for e in t if (e.isalnum() or e==' '))
    text[i] = new
text = text.str.split(' ')

Make a dictionary with each unique word as the key, and the number of times it appears as the value

In [6]:
word_dict = {}
for r in range(text.shape[0]):
    lis = text.values[r]
    for word in lis:
        if word in word_dict.keys():
            word_dict[word] +=1
        else:
            word_dict[word] = 1

Add the key, value pairs to a dictionary (after removing rt, lingering https, and hillaryclinton)

In [7]:
vals = []
remove_list = ['', ' ', 'rt', 'http','https','hillaryclinton']
keys = [ e for e in word_dict.keys() if e not in remove_list]
for key in keys:
    vals.append(word_dict.get(key))
    
tweet_df = pd.DataFrame({'words':keys, 'vals':vals})
tweet_df = tweet_df.sort_values('vals', ascending=False)
tweet_df[0:30]

,vals,words
6054,4282,to
2269,4097,the
9750,2886,is
5175,2681,a
2139,2525,you
2805,2281,of
8748,2036,for
361,1883,in
3973,1666,and
1691,1575,on


Upon first anlysis, the 30 most used words in the tweets we pulled are pretty common, with special cases such as "hillary", and "trump".  

The next strategy was to normalize each tweet for how popular it is in the English language, to find a more interesting top word count 

In [8]:
# wikiwords is a cool library that tells how often a word appears in wikipedia articles
# and works to normalize for word popularity
tweet_df['freq'] = tweet_df['words'].apply(wikiwords.freq)
tweet_df = tweet_df[tweet_df['freq'] > 1.0e-6]
tweet_df['ratio'] = tweet_df['vals'] / tweet_df['freq']
tweet_df.sort_values('ratio', ascending=False)[0:30]

,vals,words,freq,ratio
10725,968,amp,0.000004,2.649444e+08
5457,391,hypocrisy,0.000002,2.041077e+08
5895,993,trump,0.000006,1.673466e+08
7447,915,hillary,0.000006,1.660476e+08
9119,236,unfit,0.000002,1.062605e+08
4707,104,yells,0.000001,1.025306e+08
9265,290,stunning,0.000004,8.027490e+07
5694,97,yrs,0.000001,7.345456e+07
7440,115,bodyguards,0.000002,7.057417e+07
1444,82,benghazi,0.000001,6.487133e+07


This list is much more interesting, with words like Benghazi, stunning, hypocrisy, and unfit, seeming like more specific to the subject being discussed

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.

As the tweets are already loaded into a Pandas DataFrame, this part should be pretty straightforward...

In [9]:
df.sort_values('retweet_count', ascending='False')[['retweet_count', 'text']][0:10]

,retweet_count,text
0,0,@BoyGeorge @HillaryClinton Don't be sorry for ...
9659,0,@HillaryClinton get off my feed
9660,0,RT @Delo_Taylor: After 8 yrs of neglect all @P...
9661,0,Please retweet twitter blocking comments https...
9662,0,@ClintonFdn @HillaryClinton \n\nhttps://t.co/y...
9663,0,RT @Bikers4Trump: After the 3 Terrorist Attack...
9664,0,@BarbaraAResEsq @Cuckerella let's not forget @...
9665,0,RT @FoxNews: .@JudgeJeanine to @HillaryClinton...
9666,0,RT @Someweirdid: @BrianAbelTV @HillaryClinton ...
9667,0,@HillaryClinton At some point you MUST address...


Unfortunately, these tweets were pulled from the twitter streaming API, meaning we saved the tweets as they appeared, and before they were retweeted or favorited by any of the user base.

To fix this, we pulled 1/10 of them again from the REST API (not the whole set due to rate limits), saved it to a JSON, loaded it to a df, and ran the command again

In [10]:
lst = []
with open('data/update_hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lst.append(tweet)
new_df = pd.DataFrame(lst)

In [11]:
#drop duplicates is used because on retweeted tweets, twitter stores
# the retweet count from the source tweet, for all of them, so there were several repeats
new_df.sort_values('retweet_count', ascending=False)[['user', 'retweet_count', 'text']][0:100].drop_duplicates('retweet_count')[0:10]

,user,retweet_count,text
605,{'description': 'It's time to DRAIN THE SWAMP!...,17540,RT @YoungDems4Trump: Show this to those who do...
837,"{'description': '', 'name': 'GoWithItJam', 'pr...",6426,RT @bfraser747: 💥💥 BREAKING NEWS 💥💥\n\nNobody...
1106,{'description': '@HeiressLegitima @PossessGods...,2149,"RT @SenGillibrand: Love this.\n\n""@HillaryClin..."
1100,"{'name': 'CB', 'profile_sidebar_fill_color': '...",2076,"RT @Harlan: Whoa, this is a BRUTAL take on @Hi..."
615,"{'name': 'Vahe', 'profile_sidebar_fill_color':...",1804,RT @RealBenCarson: @HillaryClinton hiding her ...
402,{'description': 'The thought of a Hillary pres...,1758,RT @DrJillStein: Obama admin including @Hillar...
969,{'description': 'Patriot- Trump2016-Independen...,1587,RT @FoxNews: Fox News Poll: @realDonaldTrump l...
666,{'description': 'Beirut Vet. Former United Sta...,1586,RT @FoxNews: Fox News Poll: @realDonaldTrump l...
1131,"{'description': '', 'name': 'Life Lessons', 'p...",1392,RT @Jorge_Silva: Celebrating #HispanicHeritage...
465,"{'description': 'Deplorable Wife, Mom, Conserv...",1385,RT @ImWithYou010: @hillaryclinton talking abou...


We pulled ~1000 of our tweets from the rest API after they'd existed for a period of time, and could be retweeted. These are the 10 most popular.

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [12]:
from collections import Counter
    
def print_top_entities(label, entities, max_to_show):
    pt = PrettyTable(field_names=[label, 'Count'])
    c = Counter(entities)
    [ pt.add_row(kv) for kv in c.most_common()[:max_to_show] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

In [14]:
cursor = tweets_Collection.find()
screen_names = [ user_mention['screen_name'] 
                 for tweet in cursor
                     for user_mention in tweet['entities']['user_mentions'] ]

print_top_entities('screen name', screen_names, 10)

cursor = tweets_Collection.find()
hashtags = [ hashtag['text'] 
             for tweet in cursor
                 for hashtag in tweet['entities']['hashtags'] ]

print_top_entities('hashtag', hashtags, 10)

+-----------------+-------+
| screen name     | Count |
+-----------------+-------+
| HillaryClinton  | 15421 |
| realDonaldTrump |  2718 |
| FoxNews         |  1532 |
| POTUS           |   503 |
| CNN             |   481 |
| politico        |   283 |
| timkaine        |   263 |
| FLOTUS          |   245 |
| MSNBC           |   244 |
| USAneedsTRUMP   |   235 |
+-----------------+-------+
+-----------------------+-------+
| hashtag               | Count |
+-----------------------+-------+
| MAGA                  |   385 |
| ImWithHer             |   351 |
| SpecialReport         |   209 |
| NeverHillary          |   178 |
| DNCleak               |   177 |
| HispanicHeritageMonth |   163 |
| tcot                  |   156 |
| Trump                 |   149 |
| TrumpPence16          |   125 |
| HillaryHealth         |   102 |
+-----------------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [15]:
#definitios and config values

api = tweepy.API(auth)

MANGODB_SERVER = 'ds035816.mlab.com'
MANGODB_PORT = 35816
MANGODB_CLINTON_FF_DB_NAME = "clintonff"
MANGODB_CLINTON_FF_COLL = "collection"
db_clinton_ff_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db_clinton_ff = db_clinton_ff_client[MANGODB_CLINTON_FF_DB_NAME]
db_clinton_ff.authenticate(MANGODB_USER, MANGODB_PASS)  
 

True

In [16]:
# fetch all followers 
import time
data_followers = {}
data_followers['type'] = 'followers'
data_followers['lst'] = []

print("done")

done


In [17]:
# fetch all friends
#no need for iteration like fetching followers as number of friends is relatvely small
data_friends = {}
data_friends['type'] = 'friends'
data_friends['lst'] = []

# fetch all friends list - as this list relativly samll we won't hit the limit here.

friends_ids = []     
for itm in tweepy.Cursor(api.friends_ids, id = 'HillaryClinton').items():
    data_friends['lst'].append(itm)

# save the friend's list in mangoDB    
friends_json = json.loads(json.dumps(data_friends))  
db_clinton_ff[MANGODB_CLINTON_FF_COLL].insert_one(friends_json)

In [18]:
# Given a set of user ids, this function calls get_screen_names and plots a table of the first (max_ids) ID's and screen names.
def draw_pretty_table(ids):
    users = api.lookup_users(user_ids=ids)
    pretty_table = PrettyTable(['ID','Screen Name'])
    [pretty_table.add_row ([row.id,row.screen_name]) for row in users]
    pretty_table.align = 'l'
    print(pretty_table)

In [19]:
# print 20 firends and 20 followers data
#
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
print ("\n20 friends of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})
print ("\n20 followers of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])
    


20 friends of Hilary Clinton

+--------------------+----------------+
| ID                 | Screen Name    |
+--------------------+----------------+
| 571202103          | Medium         |
| 21337440           | ChildDefender  |
| 23449384           | amberdiscko    |
| 128790234          | Samynemir      |
| 1656913327         | sarajacobs89   |
| 325886383          | SammyKoppelman |
| 802430450          | Natasha_S_Law  |
| 729761993461248000 | ktvibbs        |
| 115740215          | SarahAudelo    |
| 34782406           | Lincoln_Ross   |
| 3044781131         | HillaryforAR   |
| 113298560          | GunaRockYa     |
| 15972271           | CdotDukes      |
| 582037089          | MiguelAyala312 |
| 734768872625188864 | AndrewBatesNC  |
| 41021335           | TroyClair      |
| 4736170399         | BrianZuzenak   |
| 150885854          | SarahPeckVA    |
| 231673             | yianni         |
| 125083946          | GillDrummond   |
+--------------------+----------------+

20 follo

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [20]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
friends = cursor[0]['lst']
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})

# we have 1 list of friends (~800 members) and over 500 list of followers; each of 5000 members,
#iterate over the followers and build the intersaction list
common_friends = []
total = 0
for follower in cursor:
    common_friends.extend(set(friends).intersection(follower['lst']))
    total += len(follower['lst'])
    
#print ('Clinton has {} followers; {} following ; {} mutal friends'.format(total, len(friends), len(common_friends)))
print("This list is not a complete one as it requires about 30 hours to fetch all followers of \n Clinto due to twitter rate limit\n we fetched about 4M out of 8M followers\n")
print ("Mutual Friends table")
draw_pretty_table(common_friends)

This list is not a complete one as it requires about 30 hours to fetch all followers of 
 Clinto due to twitter rate limit
 we fetched about 4M out of 8M followers

Mutual Friends table
+--------------------+-----------------+
| ID                 | Screen Name     |
+--------------------+-----------------+
| 3068729259         | HillaryforND    |
| 3065862285         | HillaryforNJ    |
| 172858784          | timkaine        |
| 3064597541         | HillaryforMO    |
| 3044781131         | HillaryforAR    |
| 3073392597         | HillaryforSD    |
| 3078554829         | HillaryforTX    |
| 3057577828         | HillaryforIN    |
| 3073477156         | HillaryforTN    |
| 99545933           | mouto77         |
| 3086798481         | HillaryforWV    |
| 3044746419         | HillaryforCA    |
| 3047899581         | HillaryforCT    |
| 758346398156881920 | HRCforRI        |
| 3060191026         | HillaryforMA    |
| 3057542915         | HillaryforKY    |
| 3048168670         | HillaryforDE

*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [21]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen
import json

sentiment_collection = db['sentiment']

cursor = tweets_Collection.find()

for document in cursor:
    
    screen_names = [ user_mention['screen_name']
        for user_mention in document['entities']['user_mentions'] ]
    # exclude tweets that mention both  realDonaldTrump and clinton from sentiment analysis.
    # trying to associate postive/negative in such tweets to clinton or trump is byond this scope of this case.
    if 'realDonaldTrump' in screen_names:
        continue

    # Iterate over the cleaned data - and process sentiment analysis on it. 
    url = 'https://japerk-text-processing.p.mashape.com/sentiment/' # Set destination URL here
    post_fields = {'language': 'english', 'text' : document['text']}     # Set POST fields here

    request = Request(url, urlencode(post_fields).encode())
    # I put my credit card in this website - first 40000 calls are free - DONT EXCEED 
    request.add_header('X-Mashape-Key', '1z19gbsrAwmsh1EqYI3CcBbNtjOSp1T1iVIjsnRoFmszvbDkNV')
    request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    request.add_header('Accept', 'application/json')
    data = urlopen(request).read().decode()
    datajson = json.loads(data)
    sentiment_t = {}
    if document['place']:
        place = document['place']['full_name'].split()
        sentiment_t['state'] = place[(len(place) - 1)]
    sentiment_t['text'] = document['text']
    sentiment_t['sentiment_label'] = datajson['label']
    sentiment_t['entities'] = document['entities']
    sentiment_json = json.loads(json.dumps(sentiment_t))
    sentiment_collection.insert_one(sentiment_json)

print ("done")   

HTTPError: HTTP Error 403: Forbidden

In [ ]:
# print sumber of postive/negative postive ratio per state
fields=['state', 'Postive ratio', 'Negative ratio']
pt = PrettyTable(fields)
pt.aligns = ['l' for f in fields]

states_positive_ratio = {}
sentiment_collection = db['sentiment']

for state in us_states:
    row = []
    pos_count = sentiment_collection.find({'sentiment_label': 'pos', 'state':state}).count()
    neg_count = sentiment_collection.find({'sentiment_label': 'neg', 'state':state}).count()
    pos_ratio = 0
    neg_ratio = 0
    if pos_count + neg_count > 0:
        pos_ratio = pos_count / (pos_count + neg_count) * 100
        neg_ratio = neg_count / (pos_count + neg_count) * 100
    row.append(state)
    row.append("%.2f" % pos_ratio)
    row.append("%.2f" % neg_ratio)
    states_positive_ratio[state] = pos_ratio
    pt.add_row(row)
print(pt)   

In [ ]:
import folium
import pandas as pd
from IPython.display import HTML
from base64 import b64encode
%matplotlib inline

state_geo = r'us-states.json'

sentiment_collection = db['sentiment']
states_positive_ratio = {}
for state in us_states:
    pos_count = sentiment_collection.find({'sentiment_label': 'pos', 'state':state}).count()
    neg_count = sentiment_collection.find({'sentiment_label': 'neg', 'state':state}).count()
    pos_ratio = 0
    if pos_count + neg_count > 0:
        pos_ratio = pos_count / (pos_count + neg_count) * 100
    states_positive_ratio[state] = pos_ratio

map_data = states_positive_ratio

map = folium.Map(location=(39, -100), zoom_start=4)
#map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(geo_path=state_geo, data=map_data,
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.8, line_opacity=0.6,
             reset=False)
print ("the map should be shown below; if you can't see it, please find it in DS501map.html or re-run the first cell and the cell above")
map.save("DS501map.html")
HTML(map._repr_html_())

In [ ]:
## TODO use top entites defined in 2.3
def print_top_hashtages(label, tweets, max_to_show):
    
    hashtags = [ hashtag['text'] 
         for tweet in tweets
             for hashtag in tweet['entities']['hashtags'] ]
    pt = PrettyTable(field_names=[label, 'Count'])
    c = Counter(hashtags)
    [ pt.add_row(kv) for kv in c.most_common()[:max_to_show] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

In [ ]:
# print top hastages in postive, negative and netural tweets

sentiment_collection = db['sentiment']
cursor = sentiment_collection.find({'sentiment_label': 'pos'})
print_top_hashtages('Top Hashtage in POSTIVE Tweets', cursor, 20)

cursor = sentiment_collection.find({'sentiment_label': 'neg'})
print_top_hashtages('Top Hashtagein NEGATIVE Tweets', cursor, 20)

cursor = sentiment_collection.find({'sentiment_label': 'neutral'})
print_top_hashtages('Top Hashtage in NEUTRAL Tweets', cursor, 20)


TODO elaborate more!

analyzing the hashtags distribution between positive and negative We find out that our sentiment NLP algorithm is not well trained - we need to train it again by using part of our data by classifying positive or negative for the training set based on hashtags.

due to time constraints; we decide the make another positive/negative analysis based on selection set of hashtags from the top ones.

In [ ]:
postive_hashtags = [item.lower() for item in ['ImWithHer','TurnNCBlue','StrongerTogether']]
negative_hashtags = [item.lower() for item in['WakeUpAmerica','NeverHillary','LatinosWithTrump','ImInHer','MAGA','CrookedHillary','AmericansUnitedForTrump','Im_in_Her']]

# tweet that had both postive_hashtags & negative_hashtags will be ignored.
# print top frequency words with postive_hashtags

cursor = tweets_Collection.find()
positive_tweets = []
negative_tweets = []
for document in cursor:
    hashtags = [ hashtag['text'].lower() 
         for hashtag in document['entities']['hashtags'] ]
    pos_count = len(set(postive_hashtags).intersection(hashtags))
    neg_count = len(set(negative_hashtags).intersection(hashtags))
    if (pos_count == 0 and neg_count == 0) or (pos_count > 0 and neg_count > 0):
        continue
    elif pos_count > 0:
        positive_tweets.append(document)
    else :
        negative_tweets.append(document)
        
print(len(positive_tweets))
print(len(negative_tweets))

In [ ]:
# print top hashtags for postive/negative tweets

print_top_hashtages('Top Hashtagein POSITIVE Tweets - VERSION B', positive_tweets, 20)

print_top_hashtages('Top Hashtage in NEGATIVE Tweets - VERSION B', negative_tweets, 20)


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

